In [ ]:
from odeformerplus.all import *

In [ ]:
reset_seed(42)
DEVICE = get_device()

## inference

In [ ]:
stage1 = ODEFormerPlusStage1(device=DEVICE)
load_ckpt(stage1, 'model_ckpt/stage1/316/12500.pt')
stage2 = ODEFormerPlusStage2(stage1, device=DEVICE)
load_ckpt(stage2, 'model_ckpt/stage2/316/12600.pt')

In [ ]:
model = ODEFormerPlus(stage2.encoder, stage1.decoder)
count_params(model);

In [ ]:
subexpr = lambda y: (['<mask>', '|'] * (1 + sum(s == '|' for s in y)))[:-1]
def infer_fn(x,y):
    return model.inference(
        x,
        subexpr(y),
        noise_sig=0,
        drop_rate=0,
        rescale_input=False, # already rescaled in eval_odebench
        decode_strategy='greedy',
        max_seq_len=200,
        num_beams=20,
        temperature=0.1
    )['output']
res,succ,r2 = eval_odebench(inference_fn, noise_sig=0.05, drop_rate=0.2, r2_threshold=0.5)

In [ ]:
trajs_list,sequence_list = solve_odebench(idx=10, noise_sig=0.05, drop_rate=0.2, rescale=True, print_desc=True)
trajs = trajs_list[0]
sequence = sequence_list[0]
print('True')
print('----')
SymbolicExpression().print_readable_sequence(sequence)
print('----')
pred = infer_fn(trajs, sequence)
try:
    integrate_ode_and_plot(pred, trajs, rescale_input=False, calc_r2_score=True, r2_threshold=0.9);
except: pass